In [226]:
import os
import shutil
import pandas as pd
import sorting_functions as sort
import numpy as np
import human_characterisation_functions as hcf
import plotting_funcs
import connection_parameters as con_param
import json
import math
import datetime
import shutil
import glob
import plot_intrinsic_props
import matplotlib.pyplot as plt 
import funcs_for_results_tables as get_results


In [227]:
def get_sorted_file_list (dir):
    '''
    returns sorted file list
    '''
    file_list = sorted(os.listdir(dir))
    return file_list

def get_lab_book (OP_dir):
    '''
    returns a pandas data frame of the (.xlsx) file from the OP dir
    '''
    lab_book_path = glob.glob(OP_dir + '*.xlsx' )[0]
    if lab_book_path != '~$':
        df_rec = pd.read_excel(lab_book_path, header = 1)
    return df_rec

def get_abf_files (file_list):
    filenames = []
    for file in range(len(file_list)):
        #pclamp files
        if file_list[file][-4:] == '.abf': 
            filenames.append(file_list[file])
    return filenames

def sort_protocol_names (file_list, df_rec):
    '''
    takes a file list (contents of a folder) and pandas data frame (lab notebook)
    returns indices for all recording types
    '''

    slice_indx = df_rec.index[df_rec['slice'].notnull()]
    def_slice_names = df_rec['slice'][slice_indx].tolist()
    
    index_dict = {}
    dict_keys = ['vc', 'resting', 'freq analyse', 'characterization', 'ramp',
     'spontan','vc_end', 'resting long']
    for key in dict_keys:
        index = df_rec.index[df_rec['protocol'] == key].tolist()
        #df_rec['protocol'][index] = np.nan
        index_dict[key] = index

    ic_indices = []
    for i in range(len(df_rec['protocol'])):
        name = df_rec['protocol'][i]
        if type(name) == float:
            continue
        if name.find('IC') != -1 or name.find('ic') != -1:
            ic_indices.append(i)
    index_dict['IC_files'] = ic_indices

    #other_indices = df_rec.index[df_rec['protocol'].isnull() == 0].tolist()
    return slice_indx, def_slice_names, index_dict

def fix_slice_names (def_slice_names, slice_indx):
    new_slice_names = []
    for i in range(len(def_slice_names)):
        if i < len(def_slice_names)-1:
            new_slice_names.append([def_slice_names[i]]*(slice_indx[i+1]-slice_indx[i]))
        else :
            new_slice_names.append([def_slice_names[i]]*(40))
    slice_names  = [x for xs in new_slice_names for x in xs]
    return slice_names


def get_OP_metadata (work_dir):
    file_list = get_sorted_file_list(work_dir)
    df_rec = get_lab_book(work_dir)
    filenames = get_abf_files(file_list)
    slice_indx, def_slice_names, indices_dict = sort_protocol_names(file_list, df_rec)
    slice_names = fix_slice_names(def_slice_names, slice_indx)
    return filenames, indices_dict, slice_names

def make_dir_if_not_existing(working_dir, new_dir):
    '''
    creates a dir new_dir in working_dir, if not already existing
    '''
    path = os.path.join(working_dir, new_dir)
    if os.path.isdir(path) == False: os.mkdir(path)
    return path

def to_json (work_dir, OP, file_out, out_data):
    '''
    returns a list with 3 dictionaries; 
    [1] slice names and active channels (for characterization)
    [0] con_screen_file indices and active channels
    [2] mini_meta - mini_file_indices and active chanels
    [3] treatment dictionary
    [4] connected cells in IC 
    '''
    fname = work_dir + OP + file_out
    with open(fname, "w") as outfile:
        json.dump(out_data , outfile)

def from_json (work_dir, OP, fn):
    fname = work_dir + OP + fn
    f = open(fname)
    return json.load(f)

def get_json_files (file_list):
    jsons = []
    for file in range(len(file_list)):
        #pclamp files
        if file_list[file][-5:] == '.json': 
            jsons.append(file_list[file])
    return jsons


def get_json_meta_oxitocin (work_dir, OP, file_out): # file_out = '_meta_active_chans.json'
    filenames, indices_dict, slice_names = get_OP_metadata(work_dir)

    file_list = get_sorted_file_list(work_dir)
    jsons = get_json_files(file_list)
    if OP + file_out in jsons:
        json_meta = from_json(work_dir, OP, file_out)
        return json_meta

    active_chans_all, slice_names_dict = [], []
    for i in indices_dict['vc']:
        active_channels = [int(item) for item in input('Used channels in ' + OP + ' ' + filenames[i]).split()]
        active_chans_all.append(active_channels)
        slice_names_dict.append(slice_names[i])

    vc_meta = {
        'slices' : slice_names_dict,
        'vc_files' : indices_dict['vc'],
        'active_chans_VC': active_chans_all
    }

    active_chans_all1, slice_names_dict1 = [], []
    for i in indices_dict['freq analyse']:
        active_channels = [int(item) for item in input('Used channels in ' + OP + ' ' + filenames[i]).split()]
        active_chans_all1.append(active_channels)
        slice_names_dict1.append(slice_names[i])

    active_chans_all2, slice_names_dict2 = [], []
    for i in indices_dict['characterization']:
        active_channels = [int(item) for item in input('Used channels in ' + OP + ' ' + filenames[i]).split()]
        active_chans_all2.append(active_channels)
        slice_names_dict2.append(slice_names[i])

    charact_meta = {
        'slices_freq' : slice_names_dict1,
        'freq_files' : indices_dict['freq analyse'],
        'active_chans_freq': active_chans_all1,
        'slices_char' : slice_names_dict2,
        'char_files' : indices_dict['characterization'],
        'active_chans_char': active_chans_all2
    }
    
    active_chans_all, slice_names_dict = [], []
    for i in indices_dict['resting']:
        active_channels = [int(item) for item in input('Used channels in ' + OP + ' ' + filenames[i]).split()]
        active_chans_all.append(active_channels)
        slice_names_dict.append(slice_names[i])

    resting_meta = {
        'slices_resting' : slice_names_dict,
        'resting_files' : indices_dict['resting'],
        'active_chans_resting': active_chans_all,
    }


    active_chans_all, slice_names_dict = [], []
    for i in indices_dict['ramp']:
        active_channels = [int(item) for item in input('Used channels in ' + OP + ' ' + filenames[i]).split()]
        active_chans_all.append(active_channels)
        slice_names_dict.append(slice_names[i])

    ramp_meta = {
        'slices_ramp' : slice_names_dict,
        'ramp_files' : indices_dict['ramp'],
        'active_chans_ramp': active_chans_all,
    }

    to_json(work_dir, OP, file_out, [vc_meta, charact_meta, resting_meta, ramp_meta])
    json_meta = from_json(work_dir, OP, file_out)
    return json_meta


def plot_full_RMP_trace(file_folder, files, plots_destination, channel):
    '''
    files can be a list of RMP files to be plotted one after the other
    the first one should be control and then RMP recorded during wash in
    '''
    key = 'Ch' + str(channel)
    data_plot, times_plot_all, len_times_plot = [], [], []
    for fn in files:
        filename = file_folder + fn
        data_dict = hcf.load_traces(filename)

        sweep_len = np.shape(data_dict[key][0])[0] 
        num_swps = np.shape(data_dict[key][0])[1]

        for i in range(num_swps):
            data_plot = np.append(data_plot, data_dict[key][0][:,i], 0)

        sampl_rate, units, times = hcf.get_abf_info(filename, int(key[-1]), num_swps, sweep_len)

        times_plot = np.linspace(len(times_plot_all), len(times_plot_all) + sweep_len*num_swps, sweep_len*num_swps)/sampl_rate
        len_times_plot.append(len(times_plot))
        times_plot_all = np.append(times_plot_all, times_plot)

    fig = plt.figure(figsize=(60,15))
    ax = plt.subplot(1,1,1)
    ax.plot(times_plot_all, data_plot)
    ax.vlines(times_plot_all[len_times_plot[3]], ymin = min(data_plot)  - 5, ymax = max(data_plot) + 5, lw = 1, 
        linestyle = '-', color = 'red', label = 'start wash in oxytocin')

    end_fn = filename.rfind('/') + 1

    ax.set_xlabel('time [s]', fontsize = 60)
    ax.set_ylabel(str(units)[-2:], fontsize = 60)
    ax.set_title(filename[end_fn:-4] + key, size = 20)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.figlegend(loc = 'upper right',  bbox_to_anchor=(0.95, 0.95), fontsize = 20)
    fig.patch.set_facecolor('white')
    plt.xticks(fontsize = 60) 
    plt.yticks(fontsize = 60)

    plt.savefig(plots_destination + 'all_RMP_big_change_' + filename[end_fn:-4] + '_' + key + '.png')
    plt.close()

def get_hh_mm_from_rec_time(df):
    rec_time = []
    for j in range(len(df)): 
        rec_time.append(df['recording_time'].tolist()[j][11:16])
    df['rec_time_short'] = rec_time
    return df



In [228]:

work_dir = '/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/data/mouse/2023.01.18_oxytocin/'
filenames, indices_dict, slice_names = get_OP_metadata(work_dir)
file_list = get_sorted_file_list(work_dir)

OP = '2023.01.18_oxytocin'


In [ ]:
dir_plots = sort.make_dir_if_not_existing (work_dir, 'plots')
sort.make_dir_if_not_existing (work_dir, 'data_tables')

In [ ]:
traces_folder =  os.path.join(dir_plots, "traces/")
if os.path.isdir(traces_folder) == 0 :
    for rec in range(len(filenames)):
        filename = work_dir + filenames[rec]
        plotting_funcs.plot_middle_sweep(filename)
else:
     print("skipping plotting")

In [ ]:
file_out = '_meta_active_chans.json'
jsons = get_json_files(file_list)
if OP + file_out not in jsons:
    active_chans_meta = get_json_meta_oxitocin(work_dir, OP, file_out)
else:
    active_chans_meta = from_json(work_dir, OP, file_out)


In [ ]:
df_vc = pd.DataFrame(columns=['filename', 'slice', 'cell_ch', 'recording_time',
'circumstance', 'Rs', 'Rin', 'Rs_end', 'Rin_end']) 
#circumstance: 'before', 'wash in high K', 'puff high K' 'wash out'
for i, vc in enumerate(indices_dict['vc']):

    slic = slice_names[vc]
    vc_end = indices_dict['vc_end'][i]

    filename_vc = work_dir + filenames[vc]
    filename_vc_end =  work_dir + filenames[vc_end]

    active_channels = active_chans_meta[0]['active_chans_VC'][i]
    
    rec_time = str(hcf.get_recording_time(filename_vc))
    Rs, Rin = hcf.get_access_resistance(filename_vc, active_channels)

    filename_vc = work_dir + filenames[vc]
    Rs_end, Rin_end = hcf.get_access_resistance(filename_vc_end, active_channels)

    add_df = pd.DataFrame({'filename': filenames[vc], 'slice' : slic, 'cell_ch': active_channels,
    'recording_time': rec_time, 'Rs' : Rs, 'Rin': Rin, 'Rs_end' : Rs_end, 'Rin': Rin_end })

    plotting_funcs.plot_vc_holding (filename_vc, active_channels)

    df_vc = pd.concat([df_vc.loc[:], add_df]).reset_index(drop=True)

In [ ]:
df_AP_props = pd.DataFrame(columns=['filename', 'slice', 'cell_ch', 'circumstance',
'recording_time', 'max_spikes', 'Rheobase', 'AP_heigth', 'TH', 
'max_depol', 'max_repol', 'membra_time_constant_tau', 'capacitance'])

In [ ]:
for j, char in enumerate(indices_dict['freq analyse']):
    slic = slice_names[char]

    filename_char = work_dir + filenames[char]

    active_channels = active_chans_meta[1]['active_chans_freq'][j]
    rec_time = str(hcf.get_recording_time(filename_char))

    meta_df = pd.DataFrame({'filename': filenames[char], 'slice' : slic, 'cell_ch': active_channels,
    'recording_time': rec_time })

    charact_params  = hcf.all_chracterization_params(filename_char, active_channels, 'full')
    df_char = pd.DataFrame.from_dict(charact_params)

    df_to_add = pd.concat([meta_df, df_char], axis = 1)
    df_AP_props = pd.concat([df_AP_props.loc[:], df_to_add]).reset_index(drop=True)
    plotting_funcs.plots_for_charact_file(filename_char, active_channels, 'full')


In [ ]:
inj_char = [-200,-160,-120,-80, -40, 0,40,80, 120, 160, 200, 240, 280, 320, 360, 
        400, 440, 480, 520, 560, 600]

for j, char in enumerate(indices_dict['characterization']):
    slic = slice_names[char]

    filename_char = work_dir + filenames[char]

    active_channels = active_chans_meta[1]['active_chans_char'][j]
    rec_time = str(hcf.get_recording_time(filename_char))

    meta_df = pd.DataFrame({'filename': filenames[char], 'slice' : slic, 'cell_ch': active_channels,
    'recording_time': rec_time })

    
    charact_params  = hcf.all_chracterization_params(filename_char, active_channels, inj_char)
    df_char = pd.DataFrame.from_dict(charact_params)

    df_to_add = pd.concat([meta_df, df_char], axis = 1)
    df_AP_props = pd.concat([df_AP_props.loc[:], df_to_add]).reset_index(drop=True)
    plotting_funcs.plots_for_charact_file(filename_char, active_channels, 'full')

In [ ]:
df_resting = pd.DataFrame(columns=['filename', 'slice', 'cell_ch', 'circumstance',
'recording_time', 'resting_potential'])

for k, RMP in enumerate(indices_dict['resting']):

    slic = slice_names[RMP]
    filename_resting = work_dir + filenames[RMP]
  
    active_channels = active_chans_meta[2]['active_chans_resting'][j]
    
    rec_time = str(hcf.get_recording_time(filename_resting))

    RMPs  = hcf.get_RMP_over_time(filename_resting, active_channels)

    df_to_add = pd.DataFrame({'filename': filenames[RMP], 'slice' : slic, 'cell_ch': active_channels,
    'recording_time': rec_time, 'resting_potential': RMPs })

    df_resting = pd.concat([df_resting.loc[:], df_to_add]).reset_index(drop=True)

    #plot_intrinsic_props.plot_spiking_when_RMP_increases(filename_resting, (work_dir + 'plots/RMP_plots/'))

df_resting.sort_values(by = 'recording_time', axis=0, ascending=True, inplace=True)



In [ ]:
df_vc.to_excel(work_dir + 'data_tables/' + OP + '_resistance.xlsx', index=False) 
df_AP_props.to_excel(work_dir + 'data_tables/' + OP + '_AP_props.xlsx', index=False) 
df_resting.to_excel(work_dir + 'data_tables/' + OP + '_RMPs_over_time.xlsx', index=False) 


In [ ]:
file_folder = work_dir
plots_destination = work_dir + 'plots/RMP_over_time/'

resting_S2 = [25, 26, 27, 28, 29, 30, 31]
files = []
for i in resting_S2:
    fn = filenames[i]
    files.append(fn)

channel = 8


In [232]:
df_ap_props = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/mouse/oxytocin_AP_props_all.xlsx')
df_ap_props = plot_intrinsic_props.get_hh_mm_from_rec_time(df_ap_props)
df_ap_props = get_hh_mm_from_rec_time(df_ap_props)

save_dir_ap_props = '/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/mouse/results_oxytocin/'
ap_props_dict = plot_intrinsic_props.dict_for_plotting()


In [224]:
def plot_ap_props(df_ap_props, ap_props_dict, save_dir):
    for param in ap_props_dict.keys():
        if param not in df_ap_props.columns :
            continue

        fig, ax = plt.subplots(1,1, sharex = True)
        fig.set_figheight(8)
        fig.set_figwidth(8)
        colors = ['moccasin', 'red', 'moccasin', 'cadetblue', 'mediumpurple', 'g', 'm', 'c', 'tab:brown', 'tab:olive']

        df_control = df_ap_props[df_ap_props['circumstance'] == 'ctrl']
        x_ctrl = np.linspace(0.3 , 0.9, len(df_control))
        y = df_control[param].tolist()
        ax.scatter(x_ctrl,y, color = 'darkblue')
        #ax.set_xticks(ticks = x_ctrl, labels = df_control['rec_time_short'])

        for c, cell in enumerate(df_ap_props['cell_ID'].unique()):
            if cell == 'control':
                continue
            df_cell = df_ap_props[df_ap_props['cell_ID'] == cell]

            plt_lines = []
            for i, condition in enumerate(['before', 'after']):
                df_condition = df_cell[df_cell['circumstance'] == condition]

                x = np.linspace(1.3 + i, 2 + i, len(df_condition))
                y = df_condition[param].tolist()
                ax.scatter(x,y, color = colors[c])
                plt_lines.append([x,y])
        
            x = [plt_lines[0][0][0], plt_lines[1][0][0]] 
            y = [plt_lines[0][1][0], plt_lines[1][1][0]]
            ax.plot(x, y, '-', color = 'grey', alpha = 0.5)

        for i, condition in enumerate(df_ap_props['circumstance'].unique()):
            med = df_ap_props[param][df_ap_props['circumstance'] == condition].median()
            ax.plot([x_ctrl[0] + i, x_ctrl[1] + i], [med, med], color = 'k', marker = '_')

        #ax.set_xlabel('condition')
        ax.set_ylabel(ap_props_dict[param][1])
        ax.set_xticks(ticks = x_ctrl.tolist() + [1.3] + [2.3], 
        labels = df_control['rec_time_short'].tolist() + ['Before Oxt wash in'] + ['After wash in'], rotation = 30 )
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        fig.patch.set_facecolor('white')
        plt.figlegend(loc = 'upper right',  bbox_to_anchor=(0.95, 0.95))
        fig.suptitle(ap_props_dict[param][0], fontsize = 19)
        plt.subplots_adjust(hspace=0.35)


        plt.savefig(save_dir + 'oxytocin_' + param + '.png')
        plt.close()

plot_ap_props(df_ap_props, ap_props_dict, save_dir_ap_props) 


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that 

In [251]:

RMPs = []
for i in range(len(df_ap_props)):
    if df_ap_props['rec_date'].tolist()[i] == df_ap_props['rec_date'].tolist()[0]:
        work_dir = '/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/data/mouse/2023.01.18_oxytocin/'
    else:
        work_dir = '/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/data/mouse/2023.01.16_oxytocin/'

    fn = df_ap_props['fn_resting'].tolist()[i]
    filename_resting = work_dir + fn
  
    active_chan = [df_ap_props['cell_ch'].tolist()[i]]

    RMP  = hcf.get_RMP_over_time(filename_resting, active_chan)
    RMPs.append(RMP[0][0])

In [254]:
df_ap_props['resting_potential'] = RMPs
df_ap_props.to_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/mouse/oxytocin_AP_props_all.xlsx')

In [257]:
param = 'resting_potential'

fig, ax = plt.subplots(1,1, sharex = True)
fig.set_figheight(8)
fig.set_figwidth(8)
colors = ['moccasin', 'red', 'moccasin', 'cadetblue', 'mediumpurple', 'g', 'm', 'c', 'tab:brown', 'tab:olive']

df_control = df_ap_props[df_ap_props['circumstance'] == 'ctrl']
x_ctrl = np.linspace(0.3 , 0.9, len(df_control))
y = df_control[param].tolist()
ax.scatter(x_ctrl,y, color = 'darkblue')
#ax.set_xticks(ticks = x_ctrl, labels = df_control['rec_time_short'])

for c, cell in enumerate(df_ap_props['cell_ID'].unique()):
    if cell == 'control':
        continue
    df_cell = df_ap_props[df_ap_props['cell_ID'] == cell]

    plt_lines = []
    for i, condition in enumerate(['before', 'after']):
        df_condition = df_cell[df_cell['circumstance'] == condition]

        x = np.linspace(1.3 + i, 2 + i, len(df_condition))
        y = df_condition[param].tolist()
        ax.scatter(x,y, color = colors[c])
        plt_lines.append([x,y])

    x = [plt_lines[0][0][0], plt_lines[1][0][0]] 
    y = [plt_lines[0][1][0], plt_lines[1][1][0]]
    ax.plot(x, y, '-', color = 'grey', alpha = 0.5)

for i, condition in enumerate(df_ap_props['circumstance'].unique()):
    med = df_ap_props[param][df_ap_props['circumstance'] == condition].median()
    ax.plot([x_ctrl[0] + i, x_ctrl[1] + i], [med, med], color = 'k', marker = '_')

#ax.set_xlabel('condition')
ax.set_ylabel(ap_props_dict[param][1])
ax.set_xticks(ticks = x_ctrl.tolist() + [1.3] + [2.3], 
labels = df_control['rec_time_short'].tolist() + ['Before Oxt wash in'] + ['After wash in'], rotation = 30 )
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

fig.patch.set_facecolor('white')
plt.figlegend(loc = 'upper right',  bbox_to_anchor=(0.95, 0.95))
fig.suptitle(ap_props_dict[param][0], fontsize = 19)
plt.subplots_adjust(hspace=0.35)


plt.savefig(save_dir + 'oxytocin_' + param + '.png')
plt.close()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
